In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df=pd.read_csv('US_Accidents_Dec20_Updated.csv')

# Tiền xử lý

## Xử lý với các giá trị bị thiếu.
Vì tập dữ liệu rất lớn (gần 3000000 dòng) nên em sẽ giải quyết theo 2 trường hợp sau.

- Đối với cột dữ liệu nào có tổng giá trị bị thiếu lớn hơn 250_000 thì sẽ xóa luôn cột dữ liệu đó.
    - End_Lat
    - End_Lng
    - Number
    - Wind_Chill(F) 
    - Wind_Speed(mph) 
    - Precipitation(in)
- Nếu bé hơn 250_000 thì sẽ xóa đi hàng chứa giá trị thiếu đó.
    - City (108 hàng)
    - Zipcode (1114 hàng)
    - Timezone (3430 hàng)
    - Airport_Code (6608 hàng)
    - Weather_Timestamp (46917 hàng)
    - Temperature(F) (67224 hàng)
    - Humidity(%) (71270 hàng)
    - Pressure(in) (56908 hàng)
    - Visibility(mi) (72708 hàng)
    - Wind_Direction (63474 hàng)
    - Weather_Condition (71851 hàng)
    - Sunrise_Sunset (110 hàng)
    - Civil_Twilight (110 hàng)
    - Nautical_Twilight (110 hàng)
    - Astronomical_Twilight (110 hàng)

In [3]:
data_df=data_df.drop(['End_Lat','End_Lng','Number','Wind_Chill(F)','Wind_Speed(mph)','Wind_Speed(mph)','Precipitation(in)'],axis=1).dropna()
data_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,Side,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,Tanner Rd,R,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,Providence Branch Ln,R,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,1.4,Stationary traffic on CA-17 from Summit Rd (CA...,Santa Cruz Hwy,R,...,False,False,False,False,False,False,Night,Night,Night,Night
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,0.0,Accident on US-395 Southbound at Topsy Ln.,US Highway 395 S,R,...,False,False,False,False,True,False,Day,Day,Day,Day
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,0.0,Accident on I-595 Westbound at Exit 4 / Pine I...,I-595 W,R,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
data_df.shape

(2801232, 41)

## Xem xét các cột dữ liệu.

Ở phần trên, ta thấy các cột Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop đều dùng để chú thích cho những vị trí đặc biệt trong vụ tai nạn. Quan sát thêm về các cột này, ta thấy giá trị True chiếm rất ít. Vì vậy, để làm gọn cho tập dữ liệu, em quyết định gom các cột này lại thành 1 cột chung POI (Point of Interest)

In [5]:
col_list=['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']

In [6]:
data_df['POI']=data_df[col_list].sum(axis=1).astype(bool)

In [7]:
data_df['POI'].value_counts()

False    1972573
True      828659
Name: POI, dtype: int64

In [8]:
data_df=data_df.drop(col_list,axis=1)

Ở phần trên, ta thấy các cột Sunrise_Sunset, Civil_Twilight, Nautical_Twilight, Astronomical_Twilight đều có ý nghĩa là thời gian trong ngày (Ngày/Đêm). Điểm khác nhau duy nhất là góc độ mặt trời so với đường chân trời để tính thời gian ban đêm. 
Ta có thể nghĩ tới việc chỉ lấy duy nhất cột Sunrise_Sunset để làm mốc.

In [9]:
data_df=data_df.drop(['Civil_Twilight','Nautical_Twilight','Astronomical_Twilight'],axis=1)

Đối với cột Country, do dữ liệu của nước Mỹ nên cột chỉ có 1 giá trị US -> Không có ý nghĩa -> xóa

In [22]:
data_df=data_df.drop('Country',axis=1)

Xem bộ dữ liệu sau khi xử lý

In [10]:
data_df.shape

(2801232, 26)

## Chuyển đổi kiểu dữ liệu.


### Chuyển các cột Start_Time, End_Time về kiểu dữ liệu datetime

In [13]:
data_df['Start_Time']=pd.to_datetime(data_df['Start_Time'],format='%Y-%m-%d\n%H:%M:%S',errors='coerce')
data_df['End_Time']=pd.to_datetime(data_df['End_Time'],format='%Y-%m-%d\n%H:%M:%S',errors='coerce')

In [16]:
data_df=data_df.dropna()

In [25]:
data_df.shape

(2768313, 25)